In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType

customers_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("date_of_birth", DateType(), True),
    StructField("telephone", StringType(), True),
    StructField("email", StringType(), True),
    StructField("member_since", DateType(), True),
    StructField("created_timestamp", TimestampType(), True)])

In [0]:
customers_df = (spark.readStream
    .format("json")
    .schema(customers_schema)
    .load('/Volumes/gizmobox/landing/operational_data/customers_stream/'))

In [0]:
from pyspark.sql.functions import col, current_timestamp

customers_transformed_df = (customers_df.withColumn("file_path",col("_metadata.file_path"))
.withColumn("ignestion_date", current_timestamp()))

In [0]:
streaming_query = (customers_transformed_df.writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream")
    .toTable("gizmobox.bronze.customers_stream"))


In [0]:
streaming_query.stop()

In [0]:
%sql
select *
from gizmobox.bronze.customers_stream